## 改寫原始 tutorial 檔 blockchain (1).ipynb 的 mine() 

mine() 傳回 nonce 就好，不要多做。

## Whats a Block?

![alt text](fig1.png)


## Whats a Blockchain? 

![alt text](fig2.png)


## Show Demo


## 4 Step Process!
1. Import dependencies
2. Create a block
3. Create a blockchain
4. Print out the blockchain

In [1]:
import peforth

In [2]:
%%f 擴充、修訂一下 peforth 的行為模式，讓它認得 jupyter notebook 下的 globals. Dot . 也改寫了，適合 jupyter notebook 學習環境使用。
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: ... tib. ; // ( tos -- ) A better dot that also prints the entire command line
  /// For experiments that need to show both question and result.
  /// "" . prints the command line only, w/o the TOS.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code # print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 
0 value locals // ( -- dict ) local variables at the breakpoint
\ peforth.ok('bp>',loc=locals(),cmd=':> [0] to locals')
\ *debug* init>

reDef unknown


## Step 1 - Import Dependencies

In [3]:
#generates timestamps
import datetime
#contains hashing algorithms
import hashlib

## Step 2 - Create a Block 

In [4]:
#defining the 'block' data structure
class Block:
    #each block has 7 attributes 
  
    #1 number of the block
    blockNo = 0
    #2 what data is stored in this block?
    data = None
    #3 pointer to the next block
    next = None
    #4 The hash of this block (serves as a unique ID and verifies its integrity)
    #A hash is a function that converts data into a number within a certain range. 
    hash = None
    #5 A nonce is a number only used once  
    nonce = 0
    #6 store the hash (ID) of the previous block in the chain
    previous_hash = 0x0
    #7 timestamp 
    # timestamp = datetime.datetime.now()
    timestamp = datetime.datetime(1,1,1,1,1)  # 故意把時間固定，看看 hash 是否就固定了。

    #We initialize a block by storing some data in it
    def __init__(self, data):
        self.data = data

    #Function to compute 'hash' of a block
    #a hash acts as both a unique identifier
    #& verifies its integrity
    #if someone changes the hash of a block
    #every block that comes after it is changed 
    #this helps make a blockchain immutable
    def hash(self):
        #SHA-256 is a hashing algorithm that
        # generates an almost-unique 256-bit signature that represents
        # some piece of text
        h = hashlib.sha256()
        #the input to the SHA-256 algorithm
        #will be a concatenated string
        #consisting of 5 block attributes
        #the nonce, data, previous hash, timestamp, & block #
        h.update(
        str(self.nonce).encode('utf-8') +
        str(self.data).encode('utf-8') +
        str(self.previous_hash).encode('utf-8') +
        str(self.timestamp).encode('utf-8') +
        str(self.blockNo).encode('utf-8')
        )
        #returns a hexademical string
        return h.hexdigest()
      
        ## SHOW DEMO 2, change data 

    def __str__(self):
        #print out the value of a block
        return "Block Hash: " + str(self.hash()) + "\nBlockNo: " + str(self.blockNo) + "\nBlock Data: " + str(self.data) + "\nHashes: " + str(self.nonce) + "\n--------------"


## Step 3 - Create a Blockchain

In [5]:
#defining the blockchain datastructure
#consists of 'blocks' linked together
#to form a 'chain'. Thats why its called
#'blockchain'
class Blockchain:
    
    #mining difficulty
    diff = 20
    #2^32. This is the maximum number
    #we can store in a 32-bit number
    maxNonce = 2**32
    #target hash, for mining
    target = 2 ** (256-diff)  # python 真的可以處理這麼大或更大的數字。

    #generates the first block in the blockchain
    #this is called the 'genesis block'
    block = Block("Genesis")
    #sets it as the head of our blockchain
    head = block

    #adds a given block to the chain of blocks
    #the block to be added is the only parameter
    def add(self, block):
        
        #set the hash of a given block
        #as our new block's previous hash
        block.previous_hash = self.block.hash()
        #set the block # of our new block
        #as the given block's # + 1, since
        #its next in the chain
        block.blockNo = self.block.blockNo + 1

        #set the next block equal to 
        #itself. This is the new head 
        #of the blockchain
        self.block.next = block
        self.block = self.block.next

    #Determines whether or not we can add a given block to
    #the blockchain. mine(block) 真正的產出是該 block 的 nonce, nonce 對應 hash 而 hash() 每次都重算。
    def mine(self, block):
        #from 0 to 2^32 
        for n in range(self.maxNonce):
            #is the value of the given block's hash less than our target value?
            if int(block.hash(), 16) <= self.target:
                # #if it is,
                # #add the block to the chain
                # self.add(block)
                # print(block)
                # %f datetime :> datetime.now() . cr \ 每 mine 一個都要花幾秒到十幾秒的時間
                return n
            else:
                block.nonce += 1  # 如果 hash 不出來，就 nonce++ 然後 retry
        # 如果全部失敗怎麼辦？ 
        return 0  # 傳回 0 代表失敗了 mine 不出結果。
   
    ## Show demo 3 ! Mine a block

In [6]:
2**2048 # python 真的可以處理這麼大或更大的數字。

32317006071311007300714876688669951960444102669715484032130345427524655138867890893197201411522913463688717960921898019494119559150490921095088152386448283120630877367300996091750197750389652106796057638384067568276792218642619756161838094338476170470581645852036305042887575891541065808607552399123930385521914333389668342420684974786564569494856176035326322058077805659331026192708460314150258592864177116725943603718461857357598351152301645904403697613233287231227125684710820209725157101726931323469678542580656697935045997268352998638215525166389437335543602135433229604645318478604952148193555853611059596230656

## Step 4 - Print the blockchain 

In [7]:
#initialize blockchain
blockchain = Blockchain()

#mine 10 blocks
for n in range(10):
    %f n . ." : " datetime :> datetime.now() . cr
    newblock = Block("Block " + str(n+1))
    nonce = blockchain.mine(newblock)
    %f ." nonce of newblock is " nonce . cr 
    if nonce:
        blockchain.add(newblock)
    else:
        print('Failed to mine this block:\n' + newblock)
%f n 1+ . ." : " datetime :> datetime.now() . cr cr
        
    
#print out each block in the blockchain
while blockchain.head != None:
    print(blockchain.head)
    blockchain.head = blockchain.head.next

0: 2019-01-07 19:14:48.375221
nonce of newblock is 660752
1: 2019-01-07 19:14:52.880673
nonce of newblock is 435509
2: 2019-01-07 19:14:55.833001
nonce of newblock is 1414590
3: 2019-01-07 19:15:05.113278
nonce of newblock is 250090
4: 2019-01-07 19:15:06.606433
nonce of newblock is 1724585
5: 2019-01-07 19:15:17.744125
nonce of newblock is 4012503
6: 2019-01-07 19:15:42.990823
nonce of newblock is 1226570
7: 2019-01-07 19:15:50.304680
nonce of newblock is 3789176
8: 2019-01-07 19:16:13.195723
nonce of newblock is 1003706
9: 2019-01-07 19:16:19.118362
nonce of newblock is 64917
10: 2019-01-07 19:16:19.525131

Block Hash: 1e522fb9cf033c25854be483d9a019ff8dbb577fdf7abbbdaea4e6ef6e7372f9
BlockNo: 0
Block Data: Genesis
Hashes: 0
--------------
Block Hash: f520503fd819ab099a7bf1cb677c71b9ad42aec68e2cba2aea41f6b862776848
BlockNo: 1
Block Data: Block 1
Hashes: 660752
--------------
Block Hash: a896b90f19066f6126b78be837d0a8ddd3aff1f4b601869cfb9aa187f7815664
BlockNo: 2
Block Data: Block 2
Hash

In [ ]:
%f datetime :> datetime.now() . cr

In [ ]:
%f blockchain :> head .
%f blockchain (see)

In [ ]:
%f blockchain .members